# Get Committee Data from ProPublica's Free the Files API #  
This notebook queries ProPublica's [Free the Files API](https://projects.propublica.org/free-the-files/api) for available committee data.

For more info on ProPublica check out https://propublica.org.

In [ ]:
import pickle
import pandas as pd
import requests
import json
import time

In [ ]:
def get_ftf_list(url):

    r = requests.get(url)
    if r.status_code != 200:
        return 'Request failed'
        
    raw = json.loads(r.text)
    
    return raw


def get_ftf_committee_data(slugs):
    out = {}
    failed = []
    counter = 0
    for slug in slugs:
        r = requests.get('https://projects.propublica.org/free-the-files/committees/' + slug + '.json')
        if r.status_code != 200:
            failed.append(slug)

        raw_json = json.loads(r.text)
        values = raw_json['committee']
        out[counter] = {key: values[key] for key in values.keys() if key != 'freed_files'}

        time.sleep(1)
        counter += 1
        
    return out

### Query API for list of committees ###

In [ ]:
committees = get_ftf_list(url='https://projects.propublica.org/free-the-files/committees.json')
pickle.dump(committees,open('../data/committees.p','wb'))

In [ ]:
committees_dict = {i: committees[i]['committee'] for i in range(len(committees))}
pd.DataFrame.from_dict(committees_dict,orient='index').to_csv('../data/committees.csv')

### Query API for committee data ###

In [ ]:
slugs = [v['slug'] for v in committees_dict.values()]

In [ ]:
committee_data = get_ftf_committee_data(slugs)
pd.DataFrame.from_dict(committee_data,orient='index').to_csv('../data/committee_data.csv')

### Build dictionary of committees that file with the Federal Election Commission (FEC) ###  
These committees will be used when querying ProPublica's [Campaign Finance API](https://propublica.github.io/campaign-finance-api-docs/).

In [ ]:
fec = {k: v['fec_id'] for k,v in committee_data.items() if v['fec_id'] != None}
pickle.dump(fec,open('../pickle_jar/fec_committees.p','wb'))